In [1]:
import os
import sqlite3
import pandas as pd

from langchain_openai import ChatOpenAI 
from langchain_community.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase

In [3]:
key_file='../openai_api_key.txt'

with open(key_file, 'r') as file:
    openai_api_key=file.read()
os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
inpname='./test_dataset.csv'
df=pd.read_csv(inpname,sep=';')

dbname='custom.db' 
connection = sqlite3.connect(dbname)
tablename='CustomTable' 
df.to_sql(tablename, connection, if_exists='replace')
db = SQLDatabase.from_uri('sqlite:///'+dbname)

verbose=True

llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106")

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=verbose,
    return_intermediate_steps=False
)

/home/acrnjar/miniconda3/lib/python3.9/site-packages/langchain_community/agent_toolkits/sql/base.py:138: UserWarning: Received additional kwargs {'handle_parsing_errors': True, 'return_intermediate_steps': False} which are no longer supported.
  warnings.warn(


In [8]:
for user_inquiry in [
        "What is the average age of teachers that are married?",
        ]:
    answer=agent_executor.invoke(user_inquiry, verbose=verbose)
    #print(answer['output'])




> Entering new SQL Agent Executor chain...
I need to find the average age of teachers who are married, so I should first check the tables available in the database and then write a SQL query to retrieve the required data.
Action: sql_db_list_tables
Action Input: CustomTableI need to check the schema of the CustomTable to see if it contains the necessary information about teachers and their marital status.
Action: sql_db_schema
Action Input: CustomTable
CREATE TABLE "CustomTable" (
	"index" INTEGER, 
	"Unnamed: 0" INTEGER, 
	"Age" INTEGER, 
	"Married_YES" INTEGER, 
	"Married_NO" INTEGER, 
	"Occupation" TEXT
)

/*
3 rows from CustomTable table:
index	Unnamed: 0	Age	Married_YES	Married_NO	Occupation
0	0	22	0	1	Teacher
1	1	43	0	1	Physician
2	2	37	1	0	Teacher
*/I need to write a SQL query to calculate the average age of teachers who are married.
Action: sql_db_query
Action Input: SELECT AVG(Age) FROM CustomTable WHERE Occupation = 'Teacher' AND Married_YES = 1[(33.0,)]I now know the final